<a href="https://colab.research.google.com/github/CzJLee/DeepSig-Modulation/blob/master/deepsig_slice_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/CzJLee/DeepSig-Modulation
!cp DeepSig-Modulation/signal_processing.py signal_processing.py

Cloning into 'DeepSig-Modulation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 49 (delta 17), reused 42 (delta 12), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
import h5py
import os
import numpy as np
import signal_processing as sp
import matplotlib.pyplot as plt
import time
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
from fastprogress.fastprogress import progress_bar
from sys import getsizeof

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Get location of DEEPSIG DATASET: RADIOML 2018.01A dataset
current_working_directory = os.getcwd()
file_name = os.path.join(current_working_directory, "drive/MyDrive/DeepSig-Dataset-2018/GOLD_XYZ_OSC.0001_1024.hdf5")

In [7]:
# Load File
start = time.time()
print("Loading dataset...")
f = h5py.File(file_name, 'r')

# Get the dataset from each key 
x = f["X"] # Dataset
y = f["Y"] # Labels
z = f["Z"] # SNR Value

print(f"Dataset loaded in {time.time() - start:.6f} seconds.")

Loading dataset...
Dataset loaded in 0.396137 seconds.


In [ ]:
# It is significantly faster to reshape an array than to concatenate multiple arrays. 
# Start be creating an empty array of a known size, where the first axis is the slices of the dataset we are keeping
# Then reassign the elements of this empty array to be the slices of the larger array
# Finally, reshape the array to the final shape

print("Slicing dataset with SNR >= 0")
start = time.time()

# For our dataset, x, we want to pick out the data where the SNR is greater than or equal to zero
# There are 65,536 such samples for each of the 24 modulation types
# The datatype of the X dataset is float32. This is important.
# While the original datatype of Y is int64, use int16 to save a tiny bit of memory.
dataset_slices = np.empty((24, 65536, 1024, 2), dtype=np.float32)
labels_slices = np.empty((24, 65536, 24), dtype = np.int16)

# We can then iterate over each of the modulation types
# The numbers used in these are specifically chosen to pick out only SNR >= 0 values. 
for i in progress_bar(range(24)):
    n = 106496
    start = 40960+n*i
    end = 106496+n*i
    dataset_slices[i] = x[start:end]
    labels_slices[i] = y[start:end]

# We can then reshape this array to the final shape, which is signfically faster than concatenation
dataset = np.reshape(dataset_slices, (1572864, 1024, 2))
labels = np.reshape(labels_slices, (1572864, 24))

# Remove slices from memory
del dataset_slices
del labels_slices

print(f"Datasets created in {time.time() - start:.6f} seconds.")

# For fun, I timed the process of this method versus the old method using concatenation, and it is about 3 times faster. Plus, we get to see the progress, whereas concatenation just runs with no progress indicators. Reshaping is nearly instant. 

Slicing dataset with SNR >= 0


In [ ]:
# The next step is to convert our dataset to complex values.
# We can use our empty dataset method to speed things up to avoid concatenation
print("Converting dataset to complex values...")
start = time.time()

dataset_complex = np.empty((len(dataset), 1024), dtype = np.complex64)
for i in progress_bar(range(len(dataset))):
    # Flatten the dataset, to be in I/Q format that our signal_to_complex function expects
    dataset_complex[i] = sp.signal_to_complex(np.reshape(dataset[i], (-1, )))

print(f"Converted dataset to complex values in {time.time() - start:.6f} seconds.")

# We can then remove the original dataset from memory to save precious memory
del dataset

Converting dataset to complex values...


Converted dataset to complex values in 71.715005 seconds.


In [ ]:
# Now we need to take a FFT of the complex dataset
print("Calculating FFT of dataset...")
start = time.time()

fft_dataset = np.empty((len(dataset_complex), 1024), dtype = np.float64)
for i in progress_bar(range(len(dataset_complex))):
    fft_dataset[i] = sp.fft_signal(dataset_complex[i])

print(f"Completed FFT calculaations in {time.time() - start:.6f} seconds.")

# Remove the complex dataset from memory
del dataset_complex

Calculating FFT of dataset...


In [ ]:
# Randomly shuffle dataset
shuffler = np.random.permutation(len(labels))
fft_dataset_shuffled = np.empty(shape = np.shape(fft_dataset), dtype = fft_dataset.dtype)
for i, j in enumerate(progress_bar(shuffler)):
    fft_dataset_shuffled[i] = fft_dataset[j]
    labels_shuffled[i] = labels[j]

del fft_dataset
del labels

array([1199007,  656258, 1036552, ...,  795744, 1376078, 1142829])

In [ ]:
# Save shuffled FFT datasets
save_path = os.path.join(current_working_directory, "drive/MyDrive/DeepSig-Dataset-2018/")
np.save(os.path.join(save_path, "fft_dataset_shuffled.npy"), fft_dataset_shuffled)
np.save(os.path.join(save_path, "labels_shuffled.npy"), labels_shuffled)

In [21]:
# Build CNN Model following DeepSig model

inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dense(128)(x)
outputs = layers.Dense(24, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [22]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1024, 1)]         0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 1022, 64)          256       
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 511, 64)           0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 509, 64)           12352     
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 254, 64)           0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 252, 64)           12352     
_________________________________________________________________
max_pooling1d_37 (MaxPooling (None, 126, 64)           0   

In [23]:
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the callbacks and save the best model to a new file
callbacks = [keras.callbacks.ModelCheckpoint(
    filepath='models/cnn_test.keras',
    save_best_only=True,
    monitor='val_loss')]

In [ ]:
# Time the time it takes to train the model
start = time.time()

history = model.fit(fft_dataset_shuffled, labels_shuffled, epochs=1, batch_size = 256, validation_split = 0.3, callbacks=callbacks)

print("--- Model trained in %s seconds ---" % (time.time() - start))